In [ ]:
# To be able to close the plots
%matplotlib inline

from ipywidgets import Button, Layout

root_dir = "saved_results"

save_eval_btn = Button(description="Save evaluation results", layout=Layout(width='20%'))
save_seperatly_btn = Button(description="Save results in sub-folder", layout=Layout(width='20%'))

display(save_eval_btn, save_seperatly_btn)


def save_change_in_distribution(original: pd.DataFrame, imputed: pd.DataFrame, col_name: str, imp_method: str, gap_type: int, out_dir: str):
    # Save raw
    fig, ax = plt.subplots(figsize=(12, 6))
    hist1 = plt.hist(original[col_name], bins=200, alpha=0.65, label="Original data")
    hist2 = plt.hist(imputed[col_name], bins=200, alpha=0.65, label="Imputed data")
    plt.legend(loc="upper right")
    plt.grid(axis="both")
    plt.xlabel("Measurement value")
    plt.ylabel("Measurement frequency")
    title = f"Change in distribution of {col_name} by {imp_method} with gap type {gap_type}"
    plt.title(title)
    out_file = f"{out_dir}/{title}.png"
    plt.savefig(out_file, dpi=300)
    safe_chmod(out_file, 0o664)
    plt.close(fig)

    # Save Error
    fig, ax = plt.subplots(figsize=(12, 6))
    plt.bar(range(200), hist1[0] - hist2[0])
    plt.grid(axis="both")
    plt.ylabel("Measurement frequency")
    plt.xlabel("Distribution bins")
    title = f"Distribution error of {col_name} by {imp_method} with gap type {gap_type}"
    plt.title(title)
    out_file = f"{out_dir}/{title}.png"
    plt.savefig(out_file, dpi=300)
    plt.close(fig)
    safe_chmod(out_file, 0o664)


def safe_chmod(path: str, mode: int):
    try:
        os.chmod(path, mode)
    except:
        pass


def save_evaluation_results(btn):
    global eval_results
    btn.disabled = True
    btn.description = "Saving.."
    result_file = f"{root_dir}/Results.csv"
    all_eval_results = pd.DataFrame()
    for it in eval_results:
        all_eval_results = all_eval_results.append(it)
    unique_cols = list(all_eval_results.columns)
    unique_cols.remove("Date")
    if os.path.isfile(result_file):
        full_results = pd.read_csv(result_file, sep=';', index_col=0)
        full_results = full_results.append(all_eval_results)
        full_results.drop_duplicates(unique_cols, 'last', True)
    else:
        full_results = all_eval_results
    full_results.to_csv(result_file, sep=';')
    safe_chmod(result_file, 0o664)

    targets = full_results["Target field"].value_counts().keys()
    for target in targets:
        result_file = f"{root_dir}/Results - {target}.csv"
        target_results = full_results[full_results["Target field"] == target]
        target_results.to_csv(result_file, sep=';')
        safe_chmod(result_file, 0o664)
    btn.description = "Results saved"
    btn.disabled = False


def save_seperatly_clicked(btn):
    global eval_results, eval_plots, dfs_with_gaps, imputed_dfs, dfloader, imputer, df
    btn.disabled = True
    btn.description = "Saving in sub-folder..."
    for target, results, plots in zip(dfloader.targets, eval_results, eval_plots):
        dirname = f"{root_dir}/{imputer['title']} - {target} - {imputation_date}"
        result_file = f"{dirname}/Results.csv"
        os.makedirs(dirname, mode = 0o774, exist_ok=True)
        results.to_csv(result_file, sep=';')
        safe_chmod(result_file, 0o664)
        for i in range(len(imputed_dfs)):
            save_change_in_distribution(df, imputed_dfs[i], target, imputer['title'], i + 1, dirname)
            figfile = f"{dirname}/Gap type {i + 1}.png"
            plots[i].savefig(figfile, dpi=300)
            safe_chmod(figfile, 0o664)
            compiled_dfs = pd.DataFrame({
                    f"Gapped {target}": dfs_with_gaps[i][target].values,
                    f"Imputed {target}": imputed_dfs[i][target].values
                }, index=imputed_dfs[i].index)
            dffile = f"{dirname}/Gap type {i + 1}.csv"
            compiled_dfs.to_csv(dffile, sep=';')
            safe_chmod(dffile, 0o664)
    btn.description = "Results saved in sub-folder"
    btn.disabled = False


save_eval_btn.on_click(save_evaluation_results)
save_seperatly_btn.on_click(save_seperatly_clicked)